### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [4]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy.spatial.distance import euclidean

X = df[['age', 'weight', 'height', 'ap_lo', 'ap_hi']]
y = df['cardio']

class CustomKNNClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
    
    # Функция для подсчета Евклидова расстояния
    def _distance(self, x1, x2):
        return euclidean(x1, x2)
    
    # Обучение модели (для нашего случая достаточно запомнить X и y)
    def fit(self, X, y):
        self.X_train = X.values
        self.y_train = y.values
        
    # Предсказания
    def predict(self, X_test):
        predictions = []
        for test_point in X_test.values:
            distances = [(self._distance(test_point, train_point), label) 
                         for train_point, label in zip(self.X_train, self.y_train)]
            
            # Сортируем по расстояниям и выбираем K ближайших соседей
            sorted_distances = sorted(distances, key=lambda x: x[0])
            top_k_labels = [label for _, label in sorted_distances[:self.n_neighbors]]
            
            # Голосование большинством
            prediction = Counter(top_k_labels).most_common(1)[0][0]
            predictions.append(prediction)
        return np.array(predictions)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, log_loss

def evaluate_model(model, X, y, scoring='accuracy'):
    scores = cross_val_score(model, X, y, cv=5, scoring=scoring)
    return np.mean(scores)

best_accuracy = 0
best_f1 = 0
best_roc_auc = 0
best_logloss = float("inf")
best_n_neighbors_acc = None
best_n_neighbors_f1 = None
best_n_neighbors_roc_auc = None
best_n_neighbors_logloss = None

for n_neighbors in range(1, 21):
    model = CustomKNNClassifier(n_neighbors=n_neighbors)
    model.fit(X, y)
    
    acc_score = evaluate_model(model, X, y, scoring="accuracy")
    if acc_score > best_accuracy:
        best_accuracy = acc_score
        best_n_neighbors_acc = n_neighbors
    
    f1_score_value = evaluate_model(model, X, y, scoring=make_scorer(f1_score))
    if f1_score_value > best_f1:
        best_f1 = f1_score_value
        best_n_neighbors_f1 = n_neighbors
    
    roc_auc_score_value = evaluate_model(model, X, y, scoring=make_scorer(roc_auc_score))
    if roc_auc_score_value > best_roc_auc:
        best_roc_auc = roc_auc_score_value
        best_n_neighbors_roc_auc = n_neighbors

print(f'Лучшее число соседей по точности (Accuracy): {best_n_neighbors_acc}')
print(f'Лучшее число соседей по F1-метрике: {best_n_neighbors_f1}')
print(f'Лучшее число соседей по ROC-AUC: {best_n_neighbors_roc_auc}')

TypeError: Cannot clone object '<__main__.CustomKNNClassifier object at 0x000001B8195E86B0>' (type <class '__main__.CustomKNNClassifier'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

# Деление на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем экземпляры наших моделей
models = {
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Custom KNN': CustomKNNClassifier(best_n_neighbors_acc)}  # используем лучшее найденное число соседей

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred),
        'Log-Loss': log_loss(y_test, y_pred)}  

pd.DataFrame(results).T